#### Instalação automática das dependências


In [42]:
import sys
import subprocess

# Lista de bibliotecas necessárias
libs = ["pandas", "faker"]

# Instala somente as que estiverem ausentes
for lib in libs:
    try:
        __import__(lib)
        print(f"Biblioteca '{lib}' já está instalada.")
    except ImportError:
        print(f"Instalando '{lib}'...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", lib])
        
print("\n Todas as dependências estão prontas!")

Biblioteca 'pandas' já está instalada.
Biblioteca 'faker' já está instalada.

 Todas as dependências estão prontas!


#### Importações e Configuração

In [43]:
import os
import random
from datetime import date
import pandas as pd
from faker import Faker

# Configura Faker para nomes brasileiros
fake = Faker('pt_BR')

# Cria as pastas de saída, se não existirem
os.makedirs("dados_anonimizados", exist_ok=True)
os.makedirs("agrupamentos", exist_ok=True)

print("Ambiente configurado com sucesso!")

Ambiente configurado com sucesso!


#### Perfil Base da Estefânia

In [44]:
# Perfil fixo da Estefânia
estefania = {
    "nome": "Estefânia",
    "idade": 25,
    "sexo": "F",
    "tomou_vacina": "Sim",
    "data_vacinacao": "2021-02-16",
    "local_vacinacao": "UBS Centro"
}

estefania

{'nome': 'Estefânia',
 'idade': 25,
 'sexo': 'F',
 'tomou_vacina': 'Sim',
 'data_vacinacao': '2021-02-16',
 'local_vacinacao': 'UBS Centro'}

#### Funções de Geração de Dados

In [45]:
# Gera uma pessoa aleatória
def gerar_pessoa():
    sexo = random.choice(["M", "F"])
    nome = fake.first_name_female() if sexo == "F" else fake.first_name_male()
    return {
        "nome": nome,
        "idade": random.randint(18, 80),
        "sexo": sexo,
        "tomou_vacina": random.choice(["Sim", "Não"]),
        "data_vacinacao": fake.date_between(start_date=date(2021, 1, 1), end_date=date(2021, 12, 31)).isoformat(),
        "local_vacinacao": random.choice(["UBS Norte", "UBS Sul", "UBS Centro", "UBS Leste", "UBS Oeste"])
    }

# Gera registros similares à Estefânia
def gerar_registros_estefania(quantidade=5):
    rng = random.Random()
    registros = []
    for _ in range(quantidade):
        registros.append({
            "nome": "Estefânia",
            "idade": rng.randint(20, 30),
            "sexo": "F",
            "tomou_vacina": "Sim",
            "data_vacinacao": fake.date_between(start_date=date(2021, 2, 1), end_date=date(2021, 2, 28)).isoformat(),
            "local_vacinacao": "UBS Centro"
        })
    return registros

print("✅ Funções de geração prontas!")

✅ Funções de geração prontas!


#### Geração e Anonimização dos CSVs

In [46]:
# Gera 10 arquivos CSV anonimizado
for i in range(1, 11):
    filename_csv = f"dados_anonimizados/dados_anon_{i}.csv"
    
    # 1 registro da Estefânia + 5 similares + 14 aleatórios = 20 registros
    pessoas = [estefania] + gerar_registros_estefania(5) + [gerar_pessoa() for _ in range(14)]
    
    df = pd.DataFrame(pessoas)
    
    # Pseudonimização dos nomes
    df["nome"] = [f"Pessoa_{j+1}" for j in range(len(df))]
    df.loc[0, "nome"] = "Pessoa_1"  # garante que Estefânia seja Pessoa_1
    
    # Salva o CSV anonimizado
    df.to_csv(filename_csv, index=False, encoding="utf-8")
    
    print(f"✅ {filename_csv} criado com sucesso (Estefânia = Pessoa_1)")

✅ dados_anonimizados/dados_anon_1.csv criado com sucesso (Estefânia = Pessoa_1)
✅ dados_anonimizados/dados_anon_2.csv criado com sucesso (Estefânia = Pessoa_1)
✅ dados_anonimizados/dados_anon_3.csv criado com sucesso (Estefânia = Pessoa_1)
✅ dados_anonimizados/dados_anon_4.csv criado com sucesso (Estefânia = Pessoa_1)
✅ dados_anonimizados/dados_anon_5.csv criado com sucesso (Estefânia = Pessoa_1)
✅ dados_anonimizados/dados_anon_6.csv criado com sucesso (Estefânia = Pessoa_1)
✅ dados_anonimizados/dados_anon_7.csv criado com sucesso (Estefânia = Pessoa_1)
✅ dados_anonimizados/dados_anon_8.csv criado com sucesso (Estefânia = Pessoa_1)
✅ dados_anonimizados/dados_anon_9.csv criado com sucesso (Estefânia = Pessoa_1)
✅ dados_anonimizados/dados_anon_10.csv criado com sucesso (Estefânia = Pessoa_1)


#### Função de Agrupamento do Perfil da Estefânia

In [47]:
def agrupar_registros_estefania(arquivo_entrada, arquivo_saida):
    try:
        df = pd.read_csv(arquivo_entrada)
    except FileNotFoundError:
        print(f"⚠️ Arquivo não encontrado: {arquivo_entrada}")
        return
    
    criterios_estefania = {
        "idade": (20,30),
        "sexo": "F",
        "tomou_vacina": "Sim",
        "mes_vacinacao": 2,
        "ano_vacinacao": 2021,
        "local_vacinacao": "UBS Centro"
    }
    
    df["data_vacinacao"] = pd.to_datetime(df["data_vacinacao"], errors='coerce')
    
    filtro = (
        (df["idade"].between(*criterios_estefania["idade"])) &
        (df["sexo"] == criterios_estefania["sexo"]) &
        (df["tomou_vacina"] == criterios_estefania["tomou_vacina"]) &
        (df["data_vacinacao"].dt.month == criterios_estefania["mes_vacinacao"]) &
        (df["data_vacinacao"].dt.year == criterios_estefania["ano_vacinacao"]) &
        (df["local_vacinacao"] == criterios_estefania["local_vacinacao"])
    )
    
    grupo_estefania = df[filtro]
    
    grupo_estefania.to_csv(arquivo_saida, index=False)
    
    if not grupo_estefania.empty:
        print(f"✅ {len(grupo_estefania)} registros coincidem com o perfil da Estefânia em {arquivo_entrada}")
    else:
        print(f"⚠️ Nenhum registro correspondente em {arquivo_entrada}")


#### Processamento em Lote

In [48]:
def processar_todos_os_arquivos():
    pasta_entrada = "dados_anonimizados"
    pasta_saida = "agrupamentos"
    
    for i in range(1, 11):
        arquivo_entrada = os.path.join(pasta_entrada, f"dados_anon_{i}.csv")
        arquivo_saida = os.path.join(pasta_saida, f"grupo_estefania_{i}.csv")
        agrupar_registros_estefania(arquivo_entrada, arquivo_saida)
    
    print("\n✅ Processamento concluído para todos os arquivos!")

processar_todos_os_arquivos()


✅ 6 registros coincidem com o perfil da Estefânia em dados_anonimizados\dados_anon_1.csv
✅ 6 registros coincidem com o perfil da Estefânia em dados_anonimizados\dados_anon_2.csv
✅ 6 registros coincidem com o perfil da Estefânia em dados_anonimizados\dados_anon_3.csv
✅ 6 registros coincidem com o perfil da Estefânia em dados_anonimizados\dados_anon_4.csv
✅ 6 registros coincidem com o perfil da Estefânia em dados_anonimizados\dados_anon_5.csv
✅ 6 registros coincidem com o perfil da Estefânia em dados_anonimizados\dados_anon_6.csv
✅ 6 registros coincidem com o perfil da Estefânia em dados_anonimizados\dados_anon_7.csv
✅ 6 registros coincidem com o perfil da Estefânia em dados_anonimizados\dados_anon_8.csv
✅ 6 registros coincidem com o perfil da Estefânia em dados_anonimizados\dados_anon_9.csv
✅ 6 registros coincidem com o perfil da Estefânia em dados_anonimizados\dados_anon_10.csv

✅ Processamento concluído para todos os arquivos!


#### Verificando Resultados

In [ ]:
import glob
import pandas as pd

# Lista todos os arquivos CSV gerados na pasta "agrupamentos"
arquivos_grupos = sorted(glob.glob("agrupamentos/grupo_estefania_*.csv"))

if not arquivos_grupos:
    print("⚠️ Nenhum arquivo de agrupamento encontrado. Execute o processamento primeiro.")
else:
    print(f"✅ {len(arquivos_grupos)} arquivos de agrupamento encontrados:\n")
    
    for i, arquivo in enumerate(arquivos_grupos, start=1):
        try:
            df = pd.read_csv(arquivo)
            print(f"📄 {i}. {os.path.basename(arquivo)} — {len(df)} registros encontrados")
            
            # Exibe apenas as 3 primeiras linhas, para não poluir a saída
            print(df.head())
            print("-" * 80)
        except Exception as e:
            print(f"⚠️ Erro ao ler {arquivo}: {e}")


✅ 10 arquivos de agrupamento encontrados:

📄 1. grupo_estefania_1.csv — 6 registros encontrados


,nome,idade,sexo,tomou_vacina,data_vacinacao,local_vacinacao
0,Pessoa_1,25,F,Sim,2021-02-16,UBS Centro
1,Pessoa_2,20,F,Sim,2021-02-18,UBS Centro
2,Pessoa_3,21,F,Sim,2021-02-24,UBS Centro
3,Pessoa_4,23,F,Sim,2021-02-14,UBS Centro
4,Pessoa_5,28,F,Sim,2021-02-15,UBS Centro


--------------------------------------------------------------------------------
📄 2. grupo_estefania_10.csv — 6 registros encontrados


,nome,idade,sexo,tomou_vacina,data_vacinacao,local_vacinacao
0,Pessoa_1,25,F,Sim,2021-02-16,UBS Centro
1,Pessoa_2,24,F,Sim,2021-02-06,UBS Centro
2,Pessoa_3,27,F,Sim,2021-02-10,UBS Centro
3,Pessoa_4,27,F,Sim,2021-02-27,UBS Centro
4,Pessoa_5,29,F,Sim,2021-02-18,UBS Centro


--------------------------------------------------------------------------------
📄 3. grupo_estefania_2.csv — 6 registros encontrados


...

--------------------------------------------------------------------------------
📄 9. grupo_estefania_8.csv — 6 registros encontrados


,nome,idade,sexo,tomou_vacina,data_vacinacao,local_vacinacao
0,Pessoa_1,25,F,Sim,2021-02-16,UBS Centro
1,Pessoa_2,29,F,Sim,2021-02-21,UBS Centro
2,Pessoa_3,25,F,Sim,2021-02-22,UBS Centro
3,Pessoa_4,27,F,Sim,2021-02-17,UBS Centro
4,Pessoa_5,20,F,Sim,2021-02-12,UBS Centro


--------------------------------------------------------------------------------
📄 10. grupo_estefania_9.csv — 6 registros encontrados


,nome,idade,sexo,tomou_vacina,data_vacinacao,local_vacinacao
0,Pessoa_1,25,F,Sim,2021-02-16,UBS Centro
1,Pessoa_2,30,F,Sim,2021-02-20,UBS Centro
2,Pessoa_3,24,F,Sim,2021-02-16,UBS Centro
3,Pessoa_4,26,F,Sim,2021-02-24,UBS Centro
4,Pessoa_5,24,F,Sim,2021-02-07,UBS Centro


--------------------------------------------------------------------------------
